In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
import numpy as np #  для работы с многомерными массивами
import pandas as pd # загрузка, препроцессинг и разведочный анализ данных
import matplotlib.pyplot as plt # предоставляет множество низкоуровневых графических инструментов
import seaborn as sns # содержит больше высокоуровневых графических инструментов
sns.set(rc={'figure.figsize':(10, 4)});
#sns.set()
sns.set_palette("Set3") 

Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv('../input/bank-additional-full.csv', sep=';')

In [ ]:
# Метод head(n) предназначен для просмотра первых n строк таблицы
# Признаков довольно много, поэтому для удобства транспонируем вывод
df.head(10).T

In [ ]:
df.info() # выведем общую информацию о датасете, узнаем тип каждого признака, и есть ли в данных пропуски

Проверим наши данные на выбросы(outliers) - явно некорректные значения - , а именно **age** и **duration**.

In [ ]:
sns.boxplot(df['age']); # в годах

In [ ]:
sns.boxplot(df['duration']); # в секундах

В целом, довольно реалистичная картина. Люди от 70 до 100 лет вполне могут еще являться клиентами банка. К примеру, в Италии.  

 # Каков средний возраст холостых/незамужних клиентов?

In [ ]:
# Вычислим средний возраст холостых/незамужних клиентов
#df[(df['marital'] == 'divorced') | (df['marital'] == 'single')]['age'].mean() 
#36.5053152039555

In [ ]:
# Вычислим и выведем средний возраст

# холостых клиентов
m_single = df[df['marital'] == 'single']['age'].mean() 
print('Mean age of single clients:', m_single)

# незамужних клиентов
m_divorced = df[df['marital'] == 'divorced']['age'].mean()
print('Mean age of divorced clients:', m_divorced)

# холостых/незамужних клиентов
print('Mean age of single/divorced clients:', (m_single + m_divorced) / 2)

Для интереса найдем средный возраст клиентов, находящихся в любом семейном положении. А также посмтроим график.

In [ ]:
df.groupby('marital')['age'].mean()

In [ ]:
df.groupby('marital')['age'].mean().plot(kind='barh', figsize=(10, 5)) 
plt.ylabel('Marital')
plt.xlabel('Age')
plt.show();

# В какой день недели (признак **day_of_week**) чаще всего звонили клиентам, отказавшимся от депозита?

Корреляция между категориальными переменными не может быть измерена с помощью коэффициентов Пирсона, Спирмена и Кендалла. Коэффициенты и выводы для категориальных данных обычно строятся на основании таблиц сопряжённости (кросс-таблиц, contingency tables).

Посмотрим в какой день недели чаще всего звонили клиентам, отказавшимся от депозита?.

In [ ]:
df[df['y'] == 'no']['day_of_week'].value_counts().keys()[0] #выводит элемент, у которого max значение


Больше всего звонков с отказом было осуществлено в понедельник. Визуализируем эту информацию с помощью хитмапа.

In [ ]:
sns.heatmap(pd.crosstab(df['day_of_week'], df['y']), 
            cmap="PuRd", annot=True, cbar=True);

# Постройте инфографику по признакам **marital** и **y**. Какие выводы можно сделать?



In [ ]:
# Построим кросс-таблицу
pd.crosstab(df['y'], df['marital'])

In [ ]:
# Построим хитмап
sns.heatmap(pd.crosstab(df['marital'], df['y']), 
            cmap="RdPu", annot=True, cbar=True);

Можно заметить, что женатые клиенты офорляют депозит намного реже, чем разведенные. Так как признаки категориальные для большей информации попробуем вычислить коэффициент $\chi^2$.

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(df['y'], df['marital']))

Малое значение p-value говорит о том, что связь статистически подтверждается. 

# Имеется ли связь между наличием кредита (**default**) у клиента и результатом? Постройте инфографику.

In [ ]:
pd.crosstab(df['poutcome'], df['default'])

In [ ]:
sns.heatmap(pd.crosstab(df['default'], df['poutcome']), 
            cmap="PuBu", annot=True, cbar=True);

Вычислим $\chi^2$.

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['default'], df['poutcome']))

Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что связь между наличием кредита у клиента и результатом статистически подтверждается. 

# Визуализируйте информацию об уровне образования в зависимости от среднего возраста. Есть ли статистически значимая взаимосвязь между ними?

In [ ]:
df.pivot_table(values=['age'], index=['education'], aggfunc='mean') # сводная таблица для наглядности 

In [ ]:
df.groupby('education')['age'].mean().plot(kind='barh') 
plt.ylabel('Age')
plt.show();

In [ ]:
sns.heatmap(pd.crosstab(df['education'], df['age']), 
            cmap="RdPu", annot=False, cbar=True);

Теперь, чтобы определелить есть ли между ними статистически значимая взаимосвязь, нужно закодировать значения признака **education**, поскольку он категориальный. Для этого воспользуемся следующей библиотекой:

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.education)
df['education_le'] = le.transform(df.education)  

In [ ]:
#df.head(10).T # посмотрим

In [ ]:
from scipy.stats import pearsonr, pointbiserialr
r = pearsonr(df['education_le'], df['age']) # коэффициент корелляции Пирсона
print('Pearson correlation:', r[0], 'p-value:', r[1])

Как видим, p-значение достаточно мало, поэтому у нас нет оснований утверждать, что между уровнем образования и средним возрастом клиентов нет статистически подтверждённой взаимосвязи. (она слабая, обратная)

# Что можно сказать о связи между длительностью контакта (признак **duration**) и возрастом клиента? Найдите коэффициент корреляции.

In [ ]:
r = pearsonr(df['duration'], df['age']) 
print('Pearson correlation:', r[0], 'p-value:', r[1])

Так как p-value > типичного порогового значения, то делаем вывод о том, что взаимосвязь между длительностью контакта и возрастом статистически незначима.

# Как связаны признаки **education** и **housing**?

В этот раз нам также понадобится закодировать значения признака по той же самой причине, что и ранее. Но сделаем это другим, одним из самых простых способом:

In [ ]:
new_values = {'yes':1, 'no':0, 'unknown':-1} # обычный словарь Python
df['dummy_housing'] = df['housing'].map(new_values)
#df.tail(10).T

In [ ]:
#pd.crosstab(df['education'], df['dummy_housing'])

In [ ]:
sns.heatmap(pd.crosstab(df['education'], df['housing']), 
            cmap="PuRd", annot=True, cbar=True);

In [ ]:
#pearsonr(df['dummy_housing'], df['education_le'])

In [ ]:
pointbiserialr(df['dummy_housing'], df['education_le'])

Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) между education и housing является статистически значимой. (слабая, прямая)

# Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

# Клиенты каких профессии имеют высший уровень образования?


Найдем топ-3 таких профессии:

In [ ]:
for i in [0, 1, 2]:
    print( "Top -",i + 1, df[df['education'] == 'university.degree']['job'].value_counts().keys()[i])

# Построить инфографику по признакам job и marital? Какие выводы можно сделать?


In [ ]:
# Построим хитмап
sns.heatmap(pd.crosstab(df['job'], df['marital']), 
            cmap="BuPu", annot=True, cbar=True);

На этом этапе сложно утверждать что-либо. Попробуем вычислить коэффициент $\chi^2$.

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(df['job'], df['marital']))

Малое значение p-value говорит о том, что связь статистически подтверждается.

# Зависит ли длительность контакта от уровня образования?


In [ ]:
df.groupby('education')['duration'].mean().plot(kind='barh') 
plt.ylabel('Education level')
plt.xlabel('Call duration')
plt.show();

In [ ]:
r = pearsonr(df['duration'], df['education_le'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

Как видим, p-значение достаточно мало, поэтому у нас нет оснований утверждать, что между уровнем образования и средним возрастом клиентов нет статистически подтверждённой взаимосвязи. (она слабая, обратная)

# Выполнить творческое задание: попытаться найти признаки (“фичи”), которые будут наиболее полезными для предсказания target-переменной. Использовать любые доступные инструменты разведочного анализа (вычисления,визуализация, корреляционный анализ).

Для того, чтобы узнать, какие признаки будут наиболее полезными для предсказания target-переменной, необходимо построить матрицу корреляций Спирмена. Для этого сначала перекодируем **y**

In [ ]:
new_values = {'yes':1, 'no':0} # обычный словарь Python
df['dummy_y'] = df['y'].map(new_values)
#df.head(10).T

In [ ]:
df.corr(method='spearman')

Функция heatmap библиотеки Seaborn предоставляет удобный способ визуализации таких матриц.

In [ ]:
sns.heatmap(df.corr(method='spearman'));

Какие выводы можно сделать? Наибольшая корреляция наблюдается между переменными **dummy_y** и **duration**.
Достаточно высокой является корреляция между **dummy_y** и **previous**, а также **dummy_y** и **cons.price.idx**, **dummy_y** и **education_le**, где

**duration** - last contact duration, in seconds 

**previous** - number of contacts performed before this campaign and for this client 

**cons.price.idx** - consumer price index - monthly indicator